# BART fleet-aware train length and frequency planning

## What the hell is the problem trying to solve?

Let's put ourselves in the shoes of the "BART Manager". The way we have framed this problem is that we've got two levers to pull for every single subway line, and every single time of day:

1. Frequency: How often do we run a train? (Every 6 minutes, 15 minutes)
2. Train Length: How big is each train? (4-car or 10-car?)

The two problems we'd like to solve are:

1. The demand: We want to ensure that people don't get left behind at each station. We'll call this concept, "Unmet demand".
2. The fleet: We own a specific number of cars. If we run 10-car trains every 2 minutes on every line, we would run out of cars instantly. 


So, the goal of this paper is to mathematically find the perfect allocation of trains to run, and how long they should be, so the **fewest people get left behind** (minimize unmet demand) without using more cars than we actually have.

## What are the decision variables?

Now, the naive way to do this is to highlight that frequency and train length will be our decision variables. We'll run into a non-linear problem immediately - how would we calculate the total capacity of a specific train between two stations at a specific time? We'd have to multiply the length of the train, times the frequency – NON-LINEAR!

So, the decision variables have to be defiend cleverly. We'd say:

*For a line $l$ at a time $p$, how many trains of length $k$ should I run?*

This is breaking out the problem into a count of trains of specific lengths that we run. In tangible terms: for the RED line, in the MORNING time, let's define 

1. Bucket A: 3-car trains (min possible length)
2. Bucket B: 4-car trains 
...
10. Bucket H: 10-car trains

So, for each of these buckets (for the RED line in the MORNING), we're trying to solve how many trains should we run? Suppose the solver says 1 3-car train and 2 10-car trains. That means, we have:

1. Frequency = 1 + 2 = 3 trains per hour
2. Capacity = 1x3 + 2x10 = 23 cars worth of space (each car is 200, so this would be 23x200=4600 people)

This is how we make it linear.

This also defines our decision variables:

$f_{l, p, k}$, where:

- $l$ is the line, and it could be Red, Yellow, Blue, Green, Orange
- $p$ is the time period, and it is AM, PM, MID, EVE
- $k$ is the possible car lengths, and it is 3,4,5,6,7,8,9,10

That's a total of 160 variables.

### Unmet demand

Turns out that we need one more set of variables: unmet demand:

So, suppose the capacity from the above was 4600 people, but the total demand from the data is 5000. The way we would have set our constraint is:

Capacity >= Demand.

In our current setup: 4600 is not greater than 5000, so this is infeasible right now. If we were also at our max fleet constraint, then we would be stuck!

To avoid this situation, GPT introduced another variable called "Unmet Demand". We'll tweak the constraint to be:

Capacity + Unmet Demand >= Demand. 

Now, we tell Gurobi to not just sole for the $f$ variable above, but also to minimize the sum of unmet demand in general.

Super important – Unmet demand is basically calculate for every single station pair.

## Now, how do we use our data?

OK this part is weird... 

Our data is at what it calls the "OD" level: like SFO to OAK. However, if you wanna go from SFO to OAK, you need to go through all the stations in between. 

Now, there are also other people who need to hit the stations in between

So this entire module is going to figure out across all the different origin-destination pairs, which pairs specifically use those stations in between? The part sums that up to generate the demand for that segment

Once we have that, we don't actually care about the origin destination pair anymore. 

This process is called routing

## Finally, what about the fleet?

This one is tricky as well. 

Analogy from Gemini: suppose you're juggling balls. 1 ball remains in the air for 10 seconds. So I can get a maximum of 10 balls to get the steady rhythm going.

The key idea is knowing that the ball stays in the air for 10 seconds – that's what we'll call Round Trip time in our example. 

For a specific line at a specific time – the solver runs this:

Cars Needed = Frequency x Round Trip Time x Train Length

In the above example: suppose we set 1 3-car train and 2 10-car trains at morning for red line: the cars needed for the red line would be. Also suppose a red line train takes about 2 hours round trip

So: for the 3-car train, we'd need:

1 train / hour x 2 hours trip x 3 cars = 6 total cars

For the 10-car train, we'd need:

2 trains / hour x 2 hours trip x 10 cars = 40 total cars. 

So for that schedule we'd need 46 total cars. This works because your frequency variable is setting the number of trains that run per hour. 


#### A note that dispatching

The flaw in the current approach is that we assume that our time periods are basically isolated universes:

- Morning = Universe A.
- Afternoon = Universe B.

Our model assumes that the morning is happening independent of the afternoon, and we shouldn't care about transitioning trains from one line to the other

TBH, this is fine – since we're thinking about the strategy of how we deploy BART lines and at what frequency. The idea is that if we have enough cars to survive the worst hour of the morning rush, we should have enoguh to handle the transition into the slower afternoon period. Also, we don't run out of cars during a transition, we run out when demand is highest. This issue is a real problem, but our model simplifies this porblem away by thinking about ensuring we can service the Peak Hour within each period. 

### What are we optimizing? Lexigraphic what?

Lexicographic just means phased optimization. Split the problem into two phases.

1. Minimize the unmet demand

So, ignore our costs. Let's try and find the schedule that would leave the fewest passengers behind – so, minimize unmet demand. Call the objective value $U$ here.

2. Minimize the fleet's usage (cost)

Now, we say: assumine we leave at most $U$ passengers behind, what's the smallest number of cars that gives us the solution.

The rationale here is that: "If you just said "Minimize Unmet Demand," the solver might unnecessarily run 400 cars just to pick up one extra person, or run empty trains just because it can. Phase 2 trims the fat. It finds the leanest schedule that still provides the best possible service."